In [12]:
import dotenv
import requests
import pandas as pd
import urllib.parse

In [13]:
auth_url = 'https://accounts.spotify.com/authorize'

params = {
  'client_id': dotenv.get_variable(file_path=".env.local", key="CLIENT_ID"),
  'response_type': 'code',
  'redirect_uri': 'http://localhost:3000',
  'scope': 'user-read-private user-read-email'
}

auth_url = auth_url + '?' + urllib.parse.urlencode(params)
print(auth_url)

https://accounts.spotify.com/authorize?client_id=962aeb5559434e8a8ddb2b8c78e6c3d2&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3000&scope=user-read-private+user-read-email


In [14]:
cookie = "AQAUh4aOGP0LMcAZ0CWWDWghwyEJOQ-lqIprDNYjgtXc1r4tw0AIw3U2m--Z9o0DzSogoVoGPMBRaMvSSKghvXz2os8aRkYenofj1K2-eO6lnhRqunTuCxy6KJuB4NwrjlTkhrO7UOyUNpD1mibtDgoSc99Y1cmMoYUrM2qB1a6yx3D8zcvdrOt3E_Qj0mkbhbjxqpu7l4IOnevU7A"

In [15]:
url = 'https://accounts.spotify.com/api/token'

data = {
    'grant_type': 'authorization_code',
    'code': cookie,
    'redirect_uri': 'http://localhost:3000',
    'client_id': dotenv.get_variable(file_path=".env.local", key="CLIENT_ID"),
    'client_secret': {dotenv.get_variable(file_path=".env.local", key="CLIENT_SECRET")}
}

headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.post(url, data=data, headers=headers)

access_token = response.json()['access_token']

BQCpiywL1HjarP8suct_ZgVPQDy5rxnbldMZ8Q215bYUtfbQBfLIqyIkW8IOE-bM-SftUN8s5TrSOa31TQAKdP6yChqVionCOGfFnLktUbJwhwWYF_zPk9sryJ2afEIPNfsBimVqKFyEMhZQBVmREyvueXdfWgwjPQESI5LFHSIb1nGhpJug1yVQool_PqeoUuw7sUshe85VtmfKUfBOCLk


In [16]:
url = "https://api.spotify.com/v1/me/playlists"

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    playlists = response.json()['items']
else:
    print(f'Error: {response.status_code} - {response.json()}')

In [18]:
playlist_songs = {}

for playlist in playlists:
  song_url = f"https://api.spotify.com/v1/playlists/{playlist['id']}/tracks"
  headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
  }
  response = requests.get(song_url, headers=headers)

  if response.status_code == 200:
    playlist_songs[playlist['id']] = [song['track']['id'] for song in response.json()['items']]

In [22]:
# count how many songs there are in each playlist
playlist_song_count = {playlist: len(songs) for playlist, songs in playlist_songs.items()}
playlist_song_count

# sum up the total number of songs
total_songs = sum(playlist_song_count.values())
print(f'Total songs: {total_songs}')

Total songs: 143


In [48]:
# iterate through a dictionary called playlist_songs
songs = pd.DataFrame()

for _, song_ids in playlist_songs.items():
  for song_id in song_ids:
    track_url = f"https://api.spotify.com/v1/tracks/{song_id}"
    audio_analysis_url = f"https://api.spotify.com/v1/audio-features/{song_id}"
    headers = {
      'Authorization': f'Bearer {access_token}',
      'Content-Type': 'application/json'
    }
    track_request = requests.get(track_url, headers=headers)
    audio_analysis_request = requests.get(audio_analysis_url, headers=headers)
    
    track_response = track_request.json()
    audio_analysis_response = audio_analysis_request.json()
    row = {
      'id': track_response.get('id'),
      'popularity': track_response.get('popularity'),
      'danceability': audio_analysis_response.get('danceability'),
      'energy': audio_analysis_response.get('energy'),
      'key': audio_analysis_response.get('key'),
      'loudness': audio_analysis_response.get('loudness'),
      'mode': audio_analysis_response.get('mode'),
      'speechiness': audio_analysis_response.get('speechiness'),
      'acousticness': audio_analysis_response.get('acousticness'),
      'instrumentalness': audio_analysis_response.get('instrumentalness'),
      'liveness': audio_analysis_response.get('liveness'),
      'valence': audio_analysis_response.get('valence'),
      'tempo': audio_analysis_response.get('tempo'),
      'type': audio_analysis_response.get('type'),
      'id': audio_analysis_response.get('id'),
      'uri': audio_analysis_response.get('uri'),
      'track_href': audio_analysis_response.get('track_href'),
      'analysis_url': audio_analysis_response.get('analysis_url'),
      'duration_ms': audio_analysis_response.get('duration_ms'),
      'time_signature': audio_analysis_response.get('time_signature')
    }
    # concatentate the row to the songs dataframe with the index being the length of the songs dataframe
    songs = pd.concat([songs, pd.DataFrame(row, index=[len(songs)])])

In [49]:
# find the songs that have a type of None
print(songs[songs['type'].isnull()])

Empty DataFrame
Columns: [id, popularity, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, type, uri, track_href, analysis_url, duration_ms, time_signature]
Index: []


In [50]:
# save the songs dataframe to a parquet file
songs.to_parquet('favorite-songs.parquet')